In [0]:
#Create dataframe
df = spark.createDataFrame([(1, "John", "Cambridge"), (2, "Smith", "Oxford"), (3, "Asher", "Liverpool")],["id", "name", "location"])
df.show()

+---+-----+---------+
 id| name| location|
+---+-----+---------+
 1| John|Cambridge|
 2|Smith| Oxford|
 3|Asher|Liverpool|
+---+-----+---------+

In [0]:
#write to parquet format
df.write.format('parquet').save("/FileStore/tables/parquetdata/")

In [0]:
#write to delta
df.write.format('delta').save("/FileStore/tables/deltadata/")

If number of columns or the schema changes, you would be able to write to parquet or any other format but it would not work for delta. So delta is better, note to explicitly state delta in the command

In [0]:
#Create dataframe
df1 = spark.createDataFrame([(1, "John", "Cambridge", 123456789), (2, "Smith", "Oxford", 888888888), (3, "Asher", "Liverpool", 999999999)],["id", "name", "location", "contact"])
df1.show()

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 1| John|Cambridge|123456789|
 2|Smith| Oxford|888888888|
 3|Asher|Liverpool|999999999|
+---+-----+---------+---------+

In [0]:
#overwrite parquet file specifying the mode
df1.write.mode("overwrite").format('parquet').save("/FileStore/tables/parquetdata/")

In [0]:
#overwrite to delta -- this is for schema enforcement
df1.write.mode("overwrite").format('delta').save("/FileStore/tables/deltadata/")
# this will detect mismatch in the data

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2513112895815435> in <module> 
 1 #overwrite to delta 
 ----> 2 df1 . write . mode ( "overwrite" ) . format ( 'delta' ) . save ( "/FileStore/tables/deltadata/" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 1134 self . _jwrite . save ( ) 
 1135 else : 
 -> 1136 self . _jwrite . save ( path ) 
 1137 
 1138 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : A schema mismatch detected when writing to the Delta table (Table ID: 93daafc5-b0a4-4797-99d9-94128e2da3b8).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: long (nullable = true)
-- name: string (nullable = true)
-- location: string (nullable = true)


Data schema:
root
-- id: long (nullable = true)
-- name: string (nullable = true)
-- location: string (nullable = true)
-- contact: long (nullable = true)

 
To overwrite your schema or change partitioning, please set:
'.option("overwriteSchema", "true")'.

Note that the schema can't be overwritten when using
'replaceWhere'.

In [0]:
#If you want to overwrite the data in delta without getting the schema mismatch error use .option("mergeSchema",True). This will forcefully overwrite the data
#this is to remove schema enforcement
df1.write.mode("overwrite").format('delta').option("mergeSchema", True).save("/FileStore/tables/deltadata/")
# Be sure to get the requirements if you want the schema to change

In [0]:
#to read the data from the location
spark.read.format("delta").load("/FileStore/tables/deltadata/").show()

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 3|Asher|Liverpool|999999999|
 1| John|Cambridge|123456789|
 2|Smith| Oxford|888888888|
+---+-----+---------+---------+

In [0]:
#to append data to your delta lake
df.write.mode("append").format('delta').option("mergeSchema", True).save("/FileStore/tables/deltadata/")
#note this will keep adding to the df dataframe and contact will be null here as the df has no contact column

In [0]:
spark.read.format("delta").load("/FileStore/tables/deltadata/").show()

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 3|Asher|Liverpool|999999999|
 1| John|Cambridge|123456789|
 2|Smith| Oxford|888888888|
 3|Asher|Liverpool| null|
 3|Asher|Liverpool| null|
 3|Asher|Liverpool| null|
 1| John|Cambridge| null|
 1| John|Cambridge| null|
 1| John|Cambridge| null|
 2|Smith| Oxford| null|
 2|Smith| Oxford| null|
 2|Smith| Oxford| null|
+---+-----+---------+---------+

In [0]:
# to go back to a specific time before corrupt data or data you do not want in the table use the command
spark.read.format("delta").load("/FileStore/tables/deltadata/", timestampAsOf="2021-09-06 23:00:00").show()
# with this you should be able to restore back to a previous time.

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 3|Asher|Liverpool|999999999|
 1| John|Cambridge|123456789|
 2|Smith| Oxford|888888888|
 3|Asher|Liverpool| null|
 3|Asher|Liverpool| null|
 1| John|Cambridge| null|
 1| John|Cambridge| null|
 2|Smith| Oxford| null|
 2|Smith| Oxford| null|
+---+-----+---------+---------+

In [0]:
# to view data at the very beginning of your processing, when you created the df, use
spark.read.format("delta").load("/FileStore/tables/deltadata/", versionAsOf=0).show()
#this helps for version control. e.g version 0 has 3 columns, version 1 has 4 columns. but be sure which version you want to restore

+---+-----+---------+
 id| name| location|
+---+-----+---------+
 3|Asher|Liverpool|
 1| John|Cambridge|
 2|Smith| Oxford|
+---+-----+---------+

In [0]:
# to update a table, e.g replace null values to a dummy value, you need to create a delta table
from delta.tables import DeltaTable
dtable = DeltaTable.forPath(spark, "/FileStore/tables/deltadata/")
dtable.update("contact is null",{"contact":"111111"})

In [0]:
#convert the delta table to a delta frame
dtable.toDF().show()

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 3|Asher|Liverpool| 111111|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
 1| John|Cambridge| 111111|
 2|Smith| Oxford| 111111|
 2|Smith| Oxford| 111111|
 3|Asher|Liverpool|999999999|
 1| John|Cambridge|123456789|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
 2|Smith| Oxford|888888888|
 2|Smith| Oxford| 111111|
+---+-----+---------+---------+

In [0]:
#to update a specific value
from delta.tables import DeltaTable
dtable = DeltaTable.forPath(spark, "/FileStore/tables/deltadata/")
dtable.update("id=2",{"name":"'Sean'"}) #put string for a specific value in single quotes

In [0]:
dtable.toDF().show()

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 3|Asher|Liverpool| 111111|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
 1| John|Cambridge| 111111|
 2| Sean| Oxford| 111111|
 2| Sean| Oxford| 111111|
 3|Asher|Liverpool|999999999|
 1| John|Cambridge|123456789|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
 2| Sean| Oxford|888888888|
 2| Sean| Oxford| 111111|
+---+-----+---------+---------+

In [0]:
# to delete records
from delta.tables import DeltaTable
dtable = DeltaTable.forPath(spark, "/FileStore/tables/deltadata/")
dtable.delete("id=2")
#or dtable.delete("id in (1,2)")

In [0]:
dtable.toDF().show()

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 3|Asher|Liverpool| 111111|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
 1| John|Cambridge| 111111|
 3|Asher|Liverpool|999999999|
 1| John|Cambridge|123456789|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
+---+-----+---------+---------+

In [0]:
spark.read.format('delta').load("/FileStore/tables/deltadata/").show()

+---+-----+---------+---------+
 id| name| location| contact|
+---+-----+---------+---------+
 3|Asher|Liverpool| 111111|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
 1| John|Cambridge| 111111|
 3|Asher|Liverpool|999999999|
 1| John|Cambridge|123456789|
 3|Asher|Liverpool| 111111|
 1| John|Cambridge| 111111|
+---+-----+---------+---------+

#to create delta table using sql

In [0]:
%sql
create table if not exists mydeltadata
using delta
location "/FileStore/tables/deltadata/"

In [0]:
%sql
select * from mydeltadata

id,name,location,contact
3,Asher,Liverpool,111111
3,Asher,Liverpool,111111
1,John,Cambridge,111111
1,John,Cambridge,111111
3,Asher,Liverpool,999999999
1,John,Cambridge,123456789
3,Asher,Liverpool,111111
1,John,Cambridge,111111


to rename table

In [0]:
%sql
alter table mydeltadata rename to my_delta_data_1

In [0]:
# to read json log
spark.read.format("json").load("/FileStore/tables/deltadata/_delta_log/00000000000000000001.json").show()

+--------------------+--------------------+--------------------+--------------------+
 add| commitInfo| metaData| remove|
+--------------------+--------------------+--------------------+--------------------+
 null|{0906-221758-jell...| null| null|
 null| null|{1630967377951, {...| null|
{true, 1630968557...| null| null| null|
{true, 1630968557...| null| null| null|
{true, 1630968557...| null| null| null|
{true, 1630968557...| null| null| null|
 null| null| null|{true, 1630968557...|
 null| null| null|{true, 1630968557...|
 null| null| null|{true, 1630968557...|
 null| null| null|{true, 1630968557...|
+--------------------+--------------------+--------------------+--------------------+

In [0]:
#Create dataframe for slow changing dimension type 1
df = spark.createDataFrame([(1, "John", "Cambridge"), (2, "Smith", "Oxford"), (3, "Asher", "Liverpool")],["id", "name", "location"])
df.show()

+---+-----+---------+
 id| name| location|
+---+-----+---------+
 1| John|Cambridge|
 2|Smith| Oxford|
 3|Asher|Liverpool|
+---+-----+---------+

In [0]:
#this is capture history the first time
df.write.format('delta').save("/FileStore/tables/scdtarget/")

In [0]:
#note you do not need to import libs over again if you have already imported it
from delta.tables import DeltaTable
targetdata = DeltaTable.forPath(spark, "/FileStore/tables/scdtarget/")

In [0]:
targetdata.toDF().show()

+---+-----+---------+
 id| name| location|
+---+-----+---------+
 3|Asher|Liverpool|
 1| John|Cambridge|
 2|Smith| Oxford|
+---+-----+---------+

In [0]:
# for example you get data the next day, john has move and a new record Jane has be added

df11 = spark.createDataFrame([(1, "John", "Manchester"), (4, "Jane", "London")], ["id", "name", "location"])

In [0]:
#using delta to implement scd1
df11.write.format('delta').save("/FileStore/tables/sourcedata/")

In [0]:
#view updated data in the source folder
updateddata = spark.read.format('delta').load("/FileStore/tables/sourcedata/")
updateddata.show()

+---+----+----------+
 id|name| location|
+---+----+----------+
 1|John|Manchester|
 4|Jane| London|
+---+----+----------+

In [0]:
# use merge command to update the records in the target data for scd
#in this example, you know John's address has changed, so you use the WhenMattchedUpdate(set={columnname}) and WhenNotMatchedInsert(values={columnnames}) command
from pyspark.sql.functions import *
from delta.tables import DeltaTable

targetdata = DeltaTable.forPath(spark, "/FileStore/tables/scdtarget/")
targetdata.alias("a").merge(updateddata.alias("b"),col("a.id") == col("b.id")).whenMatchedUpdate(set={"location":"b.location"}).whenNotMatchedInsert(values={"id":"b.id","name":"b.name","location":"b.location"}).execute()


In [0]:
#if you want to store your history for only 7 days use vacuum
targetdata.vacuum(200)
#by default, the history is 168 hours (24*7)

Out[69]: DataFrame[]

In [0]:
targetdata.toDF().show()

+---+-----+----------+
 id| name| location|
+---+-----+----------+
 3|Asher| Liverpool|
 2|Smith| Oxford|
 1| John|Manchester|
 4| Jane| London|
+---+-----+----------+